In [1]:
import traci
import pandas as pd
import os, sys
import sumolib
from xml.dom import minidom
import os
import random

In [2]:
sumoBinary = os.getcwd()+"/sumo/bin/sumo-gui"
config = os.getcwd()+"/sumo/tools/ChicagoNetworkReduced/osm.sumocfg"
sumoCmd = [sumoBinary, "-c", config]
tools = os.getcwd()+"sumo/tools"
sys.path.append(tools)
net_file = os.getcwd()+"/sumo/tools/ChicagoNetworkReduced/osm.net.xml"

In [3]:
taxi_trips = pd.read_csv("trip_subset.csv")

In [31]:
taxi_trips.head(1)

,Trip ID,Taxi ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,...,Extras,Trip Total,Payment Type,Company,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location
0,b58a927891b41db6aab873d25e34574a8bfeb746,4dd67761f315fa120778bcce9de01d3b69439346d88ef6...,01/01/2019 01:15:00 AM,01/01/2019 01:15:00 AM,120,0.0,1.703108e+10,1.703108e+10,8.0,8.0,...,1.0,5.0,Cash,Star North Management LLC,41.902788,-87.626146,POINT (-87.6261455896 41.9027880476),41.902788,-87.626146,POINT (-87.6261455896 41.9027880476)


In [4]:
## Just use the to from lat lons
# df = taxi_trips[['Pickup Centroid Latitude', 'Pickup Centroid Longitude',
# 'Dropoff Centroid Latitude', 'Dropoff Centroid Longitude']]
df = taxi_trips

In [33]:
df.head()

,Trip ID,Taxi ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,...,Extras,Trip Total,Payment Type,Company,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location
0,b58a927891b41db6aab873d25e34574a8bfeb746,4dd67761f315fa120778bcce9de01d3b69439346d88ef6...,01/01/2019 01:15:00 AM,01/01/2019 01:15:00 AM,120,0.0,1.703108e+10,1.703108e+10,8.0,8.0,...,1.0,5.00,Cash,Star North Management LLC,41.902788,-87.626146,POINT (-87.6261455896 41.9027880476),41.902788,-87.626146,POINT (-87.6261455896 41.9027880476)
1,f0e6ef185824288621cff5f796a4936b80c181e6,84dab24222c24f99f6c543a5d02e5b47940df071fef3ed...,01/01/2019 01:00:00 PM,01/01/2019 01:15:00 PM,830,3.9,NaN,NaN,14.0,3.0,...,0.0,13.00,Cash,Flash Cab,41.968069,-87.721559,POINT (-87.7215590627 41.968069),41.965812,-87.655879,POINT (-87.6558787862 41.96581197)
2,52c5ffc89f4bdf6b830dd82ae737dc7480db9264,9149bad3504bb6966a90f2d42d8a98fdd4d8c27bb2e408...,01/01/2019 01:15:00 PM,01/01/2019 01:30:00 PM,840,0.0,NaN,NaN,33.0,8.0,...,1.0,13.50,Cash,Blue Ribbon Taxi Association Inc.,41.857184,-87.620335,POINT (-87.6203346241 41.8571838585),41.899602,-87.633308,POINT (-87.6333080367 41.899602111)
3,3e4087b9c6aa4c8fde67006a015dd25a41467c6c,01480513a7f6f4664d6cc4c41ff3043ae6ecbc8cb17404...,01/01/2019 08:45:00 PM,01/01/2019 08:45:00 PM,300,0.0,1.703132e+10,1.703108e+10,32.0,8.0,...,0.0,5.75,Cash,Blue Ribbon Taxi Association Inc.,41.884987,-87.620993,POINT (-87.6209929134 41.8849871918),41.899156,-87.626211,POINT (-87.6262105324 41.8991556134)
4,2d0bb399485e07bd7fed6aa4a19deb426f24f6f3,100dbc840e5381b47a92bfbdbc3f231bc18b28af8cba22...,01/02/2019 08:30:00 AM,01/02/2019 08:30:00 AM,540,1.2,1.703108e+10,1.703128e+10,8.0,28.0,...,0.0,7.50,Cash,24 Seven Taxi,41.892073,-87.628874,POINT (-87.6288741572 41.8920726347),41.885300,-87.642808,POINT (-87.6428084655 41.8853000224)


In [5]:
net = sumolib.net.readNet(net_file)

In [6]:
## Coordinate transformation
px = []
py = []
dx = []
dy = []
for i in df.index:
    pickup_lat = df['Pickup Centroid Latitude'].loc[i]
    pickup_lon = df['Pickup Centroid Longitude'].loc[i]
    dropoff_lat = df['Dropoff Centroid Latitude'].loc[i]
    dropoff_lon = df['Dropoff Centroid Longitude'].loc[i]
    pickup_x, pickup_y = net.convertLonLat2XY(pickup_lon, pickup_lat)
    dropoff_x, dropoff_y = net.convertLonLat2XY(dropoff_lon, dropoff_lat)
    px.append(pickup_x)
    py.append(pickup_y)
    dx.append(dropoff_x)
    dy.append(dropoff_y)

In [7]:
df['Pickup_x'] = px
df['Pickup_y'] = py
df['Dropoff_x'] = dx
df['Dropoff_y'] = dy
# df.columns = ['Pickup Centroid Latitude', 'Pickup Centroid Longitude',
#        'Dropoff Centroid Latitude', 'Dropoff Centroid Longitude', 'Pickup_x',
#        'Pickup_y', 'Dropoff_x', 'Dropoff_y']

In [8]:
df.head()


,Trip ID,Taxi ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,...,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location,Pickup_x,Pickup_y,Dropoff_x,Dropoff_y
0,b58a927891b41db6aab873d25e34574a8bfeb746,4dd67761f315fa120778bcce9de01d3b69439346d88ef6...,01/01/2019 01:15:00 AM,01/01/2019 01:15:00 AM,120,0.0,1.703108e+10,1.703108e+10,8.0,8.0,...,41.902788,-87.626146,POINT (-87.6261455896 41.9027880476),41.902788,-87.626146,POINT (-87.6261455896 41.9027880476),17272.347244,15880.436595,17272.347244,15880.436595
1,f0e6ef185824288621cff5f796a4936b80c181e6,84dab24222c24f99f6c543a5d02e5b47940df071fef3ed...,01/01/2019 01:00:00 PM,01/01/2019 01:15:00 PM,830,3.9,NaN,NaN,14.0,3.0,...,41.968069,-87.721559,POINT (-87.7215590627 41.968069),41.965812,-87.655879,POINT (-87.6558787862 41.96581197),9419.358911,23190.652083,14859.685778,22896.308074
2,52c5ffc89f4bdf6b830dd82ae737dc7480db9264,9149bad3504bb6966a90f2d42d8a98fdd4d8c27bb2e408...,01/01/2019 01:15:00 PM,01/01/2019 01:30:00 PM,840,0.0,NaN,NaN,33.0,8.0,...,41.857184,-87.620335,POINT (-87.6203346241 41.8571838585),41.899602,-87.633308,POINT (-87.6333080367 41.899602111),17717.734886,10813.627308,16675.653209,15531.070383
3,3e4087b9c6aa4c8fde67006a015dd25a41467c6c,01480513a7f6f4664d6cc4c41ff3043ae6ecbc8cb17404...,01/01/2019 08:45:00 PM,01/01/2019 08:45:00 PM,300,0.0,1.703132e+10,1.703108e+10,32.0,8.0,...,41.884987,-87.620993,POINT (-87.6209929134 41.8849871918),41.899156,-87.626211,POINT (-87.6262105324 41.8991556134),17685.426993,13900.945181,17264.016809,15477.175148
4,2d0bb399485e07bd7fed6aa4a19deb426f24f6f3,100dbc840e5381b47a92bfbdbc3f231bc18b28af8cba22...,01/02/2019 08:30:00 AM,01/02/2019 08:30:00 AM,540,1.2,1.703108e+10,1.703128e+10,8.0,28.0,...,41.892073,-87.628874,POINT (-87.6288741572 41.8920726347),41.885300,-87.642808,POINT (-87.6428084655 41.8853000224),17037.308153,14692.385494,15875.713054,13949.005637


In [10]:
startEdge = []
endEdge = []
delta = 100
for i in df.index:
    start_x = df['Pickup_x'].loc[i]
    start_y = df['Pickup_y'].loc[i]
    end_x = df['Dropoff_x'].loc[i]
    end_y = df['Dropoff_y'].loc[i]
    startPos = [start_x, start_y]
    endPos = [end_x, end_y]
    out = sumolib.route.mapTrace([startPos, endPos], net, delta = delta)
    if len(out) == 2:
        startEdge.append(out[0].getID())
        endEdge.append(out[1].getID())
    else:
        startEdge.append(None)
        endEdge.append(None)


In [11]:
df['start_edge'] = startEdge
df['end_edge'] = endEdge

In [12]:
df2 = df.copy()

In [40]:
df = df.dropna()

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 793 entries, 3 to 1660
Data columns (total 29 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Trip ID                     793 non-null    object 
 1   Taxi ID                     793 non-null    object 
 2   Trip Start Timestamp        793 non-null    object 
 3   Trip End Timestamp          793 non-null    object 
 4   Trip Seconds                793 non-null    int64  
 5   Trip Miles                  793 non-null    float64
 6   Pickup Census Tract         793 non-null    float64
 7   Dropoff Census Tract        793 non-null    float64
 8   Pickup Community Area       793 non-null    float64
 9   Dropoff Community Area      793 non-null    float64
 10  Fare                        793 non-null    float64
 11  Tips                        793 non-null    float64
 12  Tolls                       793 non-null    float64
 13  Extras                      793 no

In [42]:
df.head()

,Trip ID,Taxi ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,...,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location,Pickup_x,Pickup_y,Dropoff_x,Dropoff_y,start_edge,end_edge
3,3e4087b9c6aa4c8fde67006a015dd25a41467c6c,01480513a7f6f4664d6cc4c41ff3043ae6ecbc8cb17404...,01/01/2019 08:45:00 PM,01/01/2019 08:45:00 PM,300,0.00,1.703132e+10,1.703108e+10,32.0,8.0,...,POINT (-87.6209929134 41.8849871918),41.899156,-87.626211,POINT (-87.6262105324 41.8991556134),17685.426993,13900.945181,17264.016809,15477.175148,583081972,1040118849#0
4,2d0bb399485e07bd7fed6aa4a19deb426f24f6f3,100dbc840e5381b47a92bfbdbc3f231bc18b28af8cba22...,01/02/2019 08:30:00 AM,01/02/2019 08:30:00 AM,540,1.20,1.703108e+10,1.703128e+10,8.0,28.0,...,POINT (-87.6288741572 41.8920726347),41.885300,-87.642808,POINT (-87.6428084655 41.8853000224),17037.308153,14692.385494,15875.713054,13949.005637,965387812#7,145857394#0
7,1cc632dd348fcd9ae2b10db17b10723c9cc64c93,053e2981d702d762acc06cc5d3076f16fa3e2ad28270ac...,01/03/2019 04:30:00 PM,01/03/2019 04:30:00 PM,337,0.99,1.703108e+10,1.703108e+10,8.0,8.0,...,POINT (-87.6197106717 41.8950334495),41.893216,-87.637844,POINT (-87.6378442095 41.8932163595),17799.866501,15015.586415,16294.114693,14824.863321,461207921#0,23891801#0
11,b7feccc5630fbdad764d1372cd385b1b5b50a732,a418af5d440c9260ccfd29882c6c4ea78d7269df7d5130...,01/04/2019 04:30:00 PM,01/04/2019 04:45:00 PM,480,1.00,1.703184e+10,1.703108e+10,32.0,8.0,...,POINT (-87.6327464887 41.8809944707),41.892508,-87.626215,POINT (-87.6262149064 41.8925077809),16706.997981,13464.767444,17258.266813,14739.085207,134319745#0,56720835#3
14,63feadf057d3ae9a2ad9cd95767165e31a4e0ca8,f7c942487c3ce86530856b5920cd4f28b0cff930a8bf7e...,01/05/2019 05:00:00 PM,01/05/2019 05:00:00 PM,720,2.40,1.703108e+10,1.703184e+10,8.0,32.0,...,POINT (-87.6207628651 41.8983317935),41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),17715.232392,15382.424498,16706.997981,13464.767444,-316912742,134319745#0


In [43]:
random.seed(42)
root = minidom.Document()
xml = root.createElement('routes')
root.appendChild(xml)
ride_count = 0
depart_time = 0
taxi_trip = []
for i in df.index:
    start = df['start_edge'].loc[i]
    end = df['end_edge'].loc[i]
    tripChild = root.createElement('trip')
    tripChild.setAttribute('id',f'taxi_{ride_count}')
    taxi_trip.append(f'taxi_{ride_count}')
    tripChild.setAttribute('type','veh_passenger')
    tripChild.setAttribute('depart',f"{depart_time:.2f}")
    tripChild.setAttribute('departLane','best')
    tripChild.setAttribute('from',f"{start}")
    tripChild.setAttribute('to',f"{end}")
    tripChild.setAttribute('speedFactor','1')
    xml.appendChild(tripChild)
    ride_count += 1
    depart_time += random.uniform(0,1)
df['id'] = taxi_trip
xml_str = root.toprettyxml(indent="\t")
file_path = 'taxi_trip_subset.xml'
with open(file_path, "w") as f:
    f.write(xml_str)

/var/folders/zd/t0wwttzn6rj0h_mjd4m2_vzw0000gn/T/ipykernel_34724/1823275594.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['id'] = taxi_trip


In [44]:
df.head()

,Trip ID,Taxi ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,...,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location,Pickup_x,Pickup_y,Dropoff_x,Dropoff_y,start_edge,end_edge,id
3,3e4087b9c6aa4c8fde67006a015dd25a41467c6c,01480513a7f6f4664d6cc4c41ff3043ae6ecbc8cb17404...,01/01/2019 08:45:00 PM,01/01/2019 08:45:00 PM,300,0.00,1.703132e+10,1.703108e+10,32.0,8.0,...,41.899156,-87.626211,POINT (-87.6262105324 41.8991556134),17685.426993,13900.945181,17264.016809,15477.175148,583081972,1040118849#0,taxi_0
4,2d0bb399485e07bd7fed6aa4a19deb426f24f6f3,100dbc840e5381b47a92bfbdbc3f231bc18b28af8cba22...,01/02/2019 08:30:00 AM,01/02/2019 08:30:00 AM,540,1.20,1.703108e+10,1.703128e+10,8.0,28.0,...,41.885300,-87.642808,POINT (-87.6428084655 41.8853000224),17037.308153,14692.385494,15875.713054,13949.005637,965387812#7,145857394#0,taxi_1
7,1cc632dd348fcd9ae2b10db17b10723c9cc64c93,053e2981d702d762acc06cc5d3076f16fa3e2ad28270ac...,01/03/2019 04:30:00 PM,01/03/2019 04:30:00 PM,337,0.99,1.703108e+10,1.703108e+10,8.0,8.0,...,41.893216,-87.637844,POINT (-87.6378442095 41.8932163595),17799.866501,15015.586415,16294.114693,14824.863321,461207921#0,23891801#0,taxi_2
11,b7feccc5630fbdad764d1372cd385b1b5b50a732,a418af5d440c9260ccfd29882c6c4ea78d7269df7d5130...,01/04/2019 04:30:00 PM,01/04/2019 04:45:00 PM,480,1.00,1.703184e+10,1.703108e+10,32.0,8.0,...,41.892508,-87.626215,POINT (-87.6262149064 41.8925077809),16706.997981,13464.767444,17258.266813,14739.085207,134319745#0,56720835#3,taxi_3
14,63feadf057d3ae9a2ad9cd95767165e31a4e0ca8,f7c942487c3ce86530856b5920cd4f28b0cff930a8bf7e...,01/05/2019 05:00:00 PM,01/05/2019 05:00:00 PM,720,2.40,1.703108e+10,1.703184e+10,8.0,32.0,...,41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),17715.232392,15382.424498,16706.997981,13464.767444,-316912742,134319745#0,taxi_4


In [27]:
df.to_csv("trip_subset_with_SUMO_ids.csv")